# Notebook Imports

In [1]:
import pandas as pd
import numpy as np

# Contstants

In [143]:
TRAIN_DATA_FILE = 'SpamData/02_Training/train-data.txt'
TEST_DATA_FILE = 'SpamData/02_Training/test-data.txt'

TOKEN_SPAM_PROB_FILE = 'SpamData/03_Testing/prob_spam.txt'
TOKEN_NOSPAM_PROB_FILE = 'SpamData/03_Testing/prob_nospam.txt'
TOKEN_ALL_PROB_FILE = 'SpamData/03_Testing/prob_all_tokens.txt'

TEST_FEATURE_FILE = 'SpamData/03_Testing/test-features.txt'
TEST_TARGET_FILE = 'SpamData/03_Testing/test-target.txt'



VOCAB_SIZE = 2500

# Read and load txt files from Numpy Array

In [4]:
sparse_train_data = np.loadtxt(TRAIN_DATA_FILE,delimiter=' ',dtype=int)

In [6]:
sparse_test_data = np.loadtxt(TEST_DATA_FILE,delimiter=' ',dtype=int)

In [28]:
sparse_test_data

array([[   8,    7,    1,    1],
       [   8,    8,    1,    1],
       [   8,   12,    1,    1],
       ...,
       [5793, 1132,    0,    1],
       [5793, 1265,    0,    1],
       [5793, 2017,    0,    1]])

## Create an Empty DataFrame

In [29]:
column_name = ['DOC_ID'] + ['CATEGORY'] + list(range(0, VOCAB_SIZE))

In [35]:
index_names = np.unique(sparse_train_data[:,0])
index_names

array([   0,    1,    2, ..., 5791, 5794, 5795])

In [42]:
full_train_data = pd.DataFrame(index=index_names,columns=column_name)
full_train_data.fillna(value=0,inplace=True)

In [43]:
full_train_data.head()

,DOC_ID,CATEGORY,0,1,2,3,4,5,6,7,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Create full matrix from sparse matrix

In [44]:
def make_full_matrix(sparse_matrix,nr_words,doc_index=0,word_idx=1,cat_itx=2,freq_index=3):
    """
    Form a full matrix form a sparse matrix
    """
    
    column_name = ['DOC_ID'] + ['CATEGORY'] + list(range(0, VOCAB_SIZE))
    doc_id_names = np.unique(sparse_train_data[:,0])
    full_matrix = pd.DataFrame(index=doc_id_names,columns=column_name)
    full_matrix.fillna(value=0,inplace=True)
    
    for i in range(sparse_matrix.shape[0]):
        
        doc_nr = sparse_matrix[i][doc_index]
        word_id = sparse_matrix[i][word_idx]
        label = sparse_matrix[i][cat_itx]
        occurance = sparse_matrix[i][freq_index]
        
        full_matrix.at[doc_nr, 'DOC_ID'] = doc_nr
        full_matrix.at[doc_nr,'CATEGORY'] = label
        full_matrix.at[doc_nr, word_id] = occurance
        
    full_matrix.set_index('DOC_ID',inplace=True)
    
    return full_matrix

In [45]:
%%time

full_train_data = make_full_matrix(sparse_train_data,VOCAB_SIZE)

CPU times: user 8.88 s, sys: 428 ms, total: 9.31 s
Wall time: 9.15 s


In [48]:
full_train_data.head()
#Se observa que hay más occurances en los primeras categoies ya que son las palabras más comúnes

,CATEGORY,0,1,2,3,4,5,6,7,8,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
0,1,2,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,1,2,0,1,2,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,2,0,2,0,0,3,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,3,0,0,1,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,1,2,4,2,3,1,...,0,0,0,0,0,0,0,0,0,0


# Training the naive Bayes Model

### Calculating probability of spam


In [82]:
prob_spam = full_train_data.CATEGORY.sum() / full_train_data.CATEGORY.size
print('Probability of spam is:',prob_spam)

Probability of spam is: 0.3111609367214748


### Total number of wods/tokens

In [93]:
full_train_features = full_train_data.drop('CATEGORY',1)

In [95]:
#Creamos un data Frame con todas las features i.e sin la CATEGORY (target)
full_train_features.head()

,0,1,2,3,4,5,6,7,8,9,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
0,2,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,0,1,2,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,0,2,0,0,3,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,0,0,1,0,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,1,2,4,2,3,1,1,...,0,0,0,0,0,0,0,0,0,0


In [97]:
#Sumamos las columnas
email_lengths = full_train_features.sum(axis=1)
email_lengths.shape

(4014,)

In [98]:
#Cuantas palabras o tokens hay por email
email_lengths[:5]

DOC_ID
0     50
1     76
2     87
3     76
4    137
dtype: int64

In [100]:
#Sumamos todas las palabras
total_wc = email_lengths.sum()
total_wc

445038

### Number of tokens in Spam & Ham Emails

In [110]:
#Spliteamos emial_lengths para que solo tenga de spam emails
spam_lengths = email_lengths[full_train_data.CATEGORY == 1]
spam_lengths[:5]

DOC_ID
0     50
1     76
2     87
3     76
4    137
dtype: int64

In [104]:
#Sumamos los tokens de esos spam emails
spam_wc = spam_lengths.sum()
spam_wc

194958

In [112]:
#Hacemos lo mismo con los no spam emails
ham_lengths = email_lengths[full_train_data.CATEGORY == 0]
ham_lengths[:5]

DOC_ID
1896     83
1898     65
1899    159
1900     57
1901     52
dtype: int64

In [113]:
nonspam_wc = ham_lengths.sum()
nonspam_wc

250080

### Summing the tokens ocurring in Spam

In [120]:
#Creamos un DataFrame con solo los SPAM emails
train_spam_tokes = full_train_features.loc[full_train_data.CATEGORY == 1]
train_spam_tokes.head()

,0,1,2,3,4,5,6,7,8,9,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
0,2,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,0,1,2,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,0,2,0,0,3,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,0,0,1,0,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,1,2,4,2,3,1,1,...,0,0,0,0,0,0,0,0,0,0


In [123]:
#Sumamos por row para conocer cada cuanto se repite cada token/palabra en los spam emails
summed_spam_tokens = train_spam_tokes.sum(axis=0) + 1
summed_spam_tokens.head()

0    1774
1     967
2    1341
3    2077
4    1347
dtype: int64

In [126]:
#Hacemos lo mismo con los ham
train_ham_tokes = full_train_features.loc[full_train_data.CATEGORY == 0]
summed_ham_tokens = train_ham_tokes.sum(axis=0) + 1
summed_ham_tokens.head()

0    5266
1    2497
2    2034
3     938
4    1595
dtype: int64

## P(Token | Spam) - Probability that a token occures given that it's  a spam email

In [134]:
prob_tokens_spam = summed_spam_tokens / (spam_wc + VOCAB_SIZE)
prob_tokens_spam[:5]

0    0.008984
1    0.004897
2    0.006791
3    0.010519
4    0.006822
dtype: float64

## P(Token | Ham) - Probability that a token occures given that it's  a legit email

In [136]:
prob_tokens_nospam = summed_ham_tokens / (nonspam_wc + VOCAB_SIZE)
prob_tokens_nospam[:5]

0    0.020849
1    0.009886
2    0.008053
3    0.003714
4    0.006315
dtype: float64

## P(Token)

In [141]:
prob_tokens_all = full_train_features.sum(axis=0) / total_wc
prob_tokens_all[:5]

0    0.015814
1    0.007779
2    0.007579
3    0.006770
4    0.006606
dtype: float64

## Saving the model

In [142]:
np.savetxt(TOKEN_SPAM_PROB_FILE, prob_tokens_spam)
np.savetxt(TOKEN_NOSPAM_PROB_FILE, prob_tokens_nospam)
np.savetxt(TOKEN_ALL_PROB_FILE, prob_tokens_all)

## Prepare Test Data


In [158]:
#Populamos la tabla con la test data
full_test_data = make_full_matrix(sparse_test_data,nr_words = VOCAB_SIZE)

In [163]:
X_test = full_test_data.loc[:, full_test_data.columns != 'CATEGORY']
y_test = full_test_data.CATEGORY

In [165]:
np.savetxt(TEST_FEATURE_FILE,X_test)
np.savetxt(TEST_TARGET_FILE,y_test)